In [1]:
import pyedflib
import os
import shutil
import glob
import datetime
import time
import pandas as pd
import numpy as np
import numpy as np
import heartpy as hp
import matplotlib.pyplot as plt

from numpy import mean, sqrt, square, arange
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Read raw PPG files
path = r'prof_data\resteaze\00000192-resteaze-sync.csv'
mdf = pd.read_csv(path, sep=',')
mdf.head()

,Unnamed: 0,counter,unixTimes,accelerometerX,accelerometerY,accelerometerZ,capacitorsA,capacitorsB,capacitorsC,gyroscopeX,...,sleep_state,Leg Movement,Hypopnea,Arousal,RERA,Relative Desaturation,Obstructive Apnea,Central Apnea,Leg Movement (L),Leg Movement (R)
0,2568,2569,1645771253032,1353.0,-472.0,-3934.0,30734.0,NaN,NaN,-149.0,...,-1.0,0,0,0,0,0,0,0,0,0
1,2569,2574,1645771253039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0,0,0,0,0,0,0,0,0
2,2570,2570,1645771253072,1346.0,-486.0,-3924.0,30733.0,NaN,NaN,-142.0,...,-1.0,0,0,0,0,0,0,0,0,0
3,2571,2575,1645771253079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0,0,0,0,0,0,0,0,0
4,2572,2571,1645771253112,1342.0,-471.0,-3923.0,30733.0,NaN,NaN,-171.0,...,-1.0,0,0,0,0,0,0,0,0,0


In [3]:
fname = path.split('\\')[-1].split('-')[0]

# TODO: Try to get all the data in one row

In [4]:
mdf.columns

Index(['Unnamed: 0', 'counter', 'unixTimes', 'accelerometerX',
       'accelerometerY', 'accelerometerZ', 'capacitorsA', 'capacitorsB',
       'capacitorsC', 'gyroscopeX', 'gyroscopeY', 'gyroscopeZ', 'ledIR',
       'ledRed', 'ledGreen', 'tempObject', 'tempAmbient', 'sync', 'time',
       'sleep_stage', 'sleep_state', 'Leg Movement', 'Hypopnea', 'Arousal',
       'RERA', 'Relative Desaturation', 'Obstructive Apnea', 'Central Apnea',
       'Leg Movement (L)', 'Leg Movement (R)'],
      dtype='object')

In [5]:
mdf["ledGreen"].dropna()

1         7636.0
3         7639.0
5         7639.0
7         7638.0
9         7634.0
           ...  
571943    6437.0
571945    6445.0
571947    6456.0
571949    6462.0
571951    6465.0
Name: ledGreen, Length: 283928, dtype: float64

In [6]:
temp = mdf["tempObject"].dropna().to_list()
data = []
for i in range(len(temp)):
    t = temp[i]
    for i in range(120):
        data.append(t)

In [7]:
len(data)

275280

In [8]:
# TODO: Add combined epoch data to the dataframe

In [9]:
df = mdf[mdf.sleep_state != -1].reset_index()

df = df[[
    'ledGreen',
    'sleep_state'
]].dropna()
# df['sleep_state'] = df['sleep_state'].mask(lambda col: col == -1, 6)

# Applying the condition
df["sleep_state"] = np.where(df["sleep_state"] == 0, 0, 1)
df

,ledGreen,sleep_state
1,7259.0,0
3,7262.0,0
4,7262.0,0
7,7263.0,0
9,7265.0,0
...,...,...
538764,6437.0,1
538766,6445.0,1
538768,6456.0,1
538770,6462.0,1


In [10]:
df['sleep_state'].value_counts()

0    160167
1    107288
Name: sleep_state, dtype: int64

In [11]:
from scipy.signal import butter, sosfilt, sosfreqz

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = sosfilt(sos, data)
        return y

# apply bandpass filter

fs = 25
lowcut = 0.65
highcut = 3.0

df['ledGreen'] = butter_bandpass_filter(df['ledGreen'], lowcut, highcut, fs, order=4)

In [12]:
# plt.plot(df['ledGreen'][800:1024], label='Acc signal (%g Hz)' % 25)
# plt.xlabel('time (ms)')
# plt.grid(True)
# plt.axis('tight')
# plt.legend(loc='upper left')

# plt.show()

# plt.plot(df['ledGreen'][800:1024], label='Bandpass signal (%g Hz)' % 25)
# plt.xlabel('time (ms)')
# plt.grid(True)
# plt.axis('tight')
# plt.legend(loc='upper left')

# plt.show()


# Get the analysis for epochs

In [ ]:
import heartpy as hp

#run the analysis
wd, m = hp.process(np.array(df['ledGreen'][1800:2100]), sample_rate=25.0)

#set large figure
plt.figure(figsize=(25,4))

#call plotter
hp.plotter(wd, m)

#display measures computed
for measure in m.keys():
    print('%s: %f' %(measure, m[measure]))

In [13]:
#run the segmented analysis
wd_seg, m_seg = hp.process_segmentwise(np.array(df['ledGreen']), sample_rate=25.0, segment_width = 30)


c:\Users\lucki\anaconda3\envs\tf\lib\site-packages\scipy\interpolate\fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
c:\Users\lucki\anaconda3\envs\tf\lib\site-packages\numpy\ma\core.py:5244: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
c:\Users\lucki\anaconda3\envs\tf\lib\site-packages\numpy\core\fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [14]:
hp.segment_plotter(seg_wd, seg_m, title='Heart Rate Signal Peak Detection', figsize=(25, 6), path='heartpy_outputs/{}'.format(fname), start=0, end=None, step=1)

NameError: name 'seg_wd' is not defined

In [56]:
def get_analysis_data(seg_indices, seg_data, fs=25, epoch_length=30, hasData=False):
    data = []

    for i in range(len(seg_indices)):
        tmp = seg_data[i]
        if not hasData:
            for i in range(fs * epoch_length):
                data.append(tmp)
        else:
            for i in range(len(tmp)):
                data.append(tmp[i])

    return data

fs = 25
lowcut =0.55
highcut = 3
order = 3
epoch_length = 30

# Create a dataframe the respiration rate and bpm and heart rate variability
df_seg = pd.DataFrame()
df_seg['bpm'] = get_analysis_data(wd_seg['segment_indices'], m_seg['bpm'], fs, epoch_length)
df_seg['hrv'] = get_analysis_data(wd_seg['segment_indices'], m_seg['rmssd'], fs, epoch_length)
df_seg['hr'] = get_analysis_data(wd_seg['segment_indices'], wd_seg['hr'], fs, epoch_length, True)
df_seg['breathingrate'] = get_analysis_data(wd_seg['segment_indices'], m_seg['breathingrate'] * 1000, fs, epoch_length)


In [58]:
df_seg

,bpm,hrv,hr,breathingrate
0,67.961165,163.095064,34.196952,0.161812
1,67.961165,163.095064,31.895984,0.161812
2,67.961165,163.095064,29.436163,0.161812
3,67.961165,163.095064,27.197439,0.161812
4,67.961165,163.095064,25.376856,0.161812
...,...,...,...,...
266245,65.573770,230.879326,13.706693,0.227687
266246,65.573770,230.879326,20.957312,0.227687
266247,65.573770,230.879326,28.907632,0.227687
266248,65.573770,230.879326,35.824982,0.227687


In [45]:
wd_seg['hr'][-1][-5:]

array([13.70669347, 20.95731211, 28.90763241, 35.82498188, 40.54002334])

#### Number of accepted and rejected peaks

In [ ]:
wd.keys()

In [ ]:
# Rejected peaks
rejectedpeaks = wd['removed_beats']

(rejectedpeaks).shape

In [ ]:
# Accepted peaks
peaklist = wd['peaklist']

(np.asarray(peaklist)).shape

### Update df with the analysis for epochs

In [ ]:
wd['hr'].shape

In [ ]:
df['ledGreen'].shape

In [ ]:
seg_wd.keys(), seg_m.keys()

In [ ]:
len(seg_m['bpm'])

In [ ]:
len(seg_wd['RR_list'])

In [ ]:
len(seg_wd['hr'])